# Práctico 1 - Parte 1

[Enunciado](https://github.com/DiploDatos/AprendizajeProfundo/blob/master/Practico.md) del trabajo práctico.

**Implementación de red neuronal [Perceptrón Multicapa](https://en.wikipedia.org/wiki/Multilayer_perceptron) (MLP).**

## Integrantes
- Mauricio Caggia
- Luciano Monforte
- Gustavo Venchiarutti
- Guillermo Robiglio

En esta primera parte del trabajo práctico se realiza un análisis de los datos y reducción de los mismos para facilitar el manupuleo de los mismos durante el armado de los datasets.

## Importaciones

In [1]:
import json
import pandas as pd

## Descarga de datos

### MeLi Challenge 2019 Dataset

Para el proyecto del curso vamos a usar el dataset del MeLi Challenge 2019, para clasificación automática de categorías de productos:

In [ ]:
%%bash

curl -L https://cs.famaf.unc.edu.ar/\~ccardellino/resources/diplodatos/meli-challenge-2019.tar.bz2 -o ./data/meli-challenge-2019.tar.bz2
tar jxvf ./data/meli-challenge-2019.tar.bz2 -C ./data/

## Constantes

In [4]:
ARCHIVO_SET_DE_ENTRENAMIENTO = './data/meli-challenge-2019/spanish.train.jsonl.gz'
ARCHIVO_SET_DE_PRUEBA = './data/meli-challenge-2019/spanish.test.jsonl.gz'
ARCHIVO_SET_DE_VALIDACION = './data/meli-challenge-2019/spanish.validation.jsonl.gz'
ARCHIVO_TOKENS = './data/meli-challenge-2019/spanish_token_to_index.json.gz'

## Carga de datos

Esta carga de datos se realiza con el fin de explorar los mismos. En la parte 2 de este trabajo práctico se realiza otra carga de datos para construir el dataset.

In [5]:
%%time
file_paths = [ARCHIVO_SET_DE_ENTRENAMIENTO, # Ingresar opción 0 👁 ⚠ Tarda más de 2 minutos en cargar y puede que haya un desbordamiento de RAM o muera el kernel⚠
              ARCHIVO_SET_DE_PRUEBA,  # Ingresar opción 1
              ARCHIVO_SET_DE_VALIDACION] # Ingresar opción 2 ⚠ Tarda más de 30 segundos en cargar
i = int(input('Ingresar opción para carga de archivo (0 a 2): '))
df = pd.read_json(path_or_buf=file_paths[i], lines=True)

Ingresar opción para carga de archivo (0 a 2): 2
CPU times: user 20.7 s, sys: 3.77 s, total: 24.5 s
Wall time: 26.2 s


In [ ]:
tokens = pd.read_json(path_or_buf=ARCHIVO_TOKENS, lines=True).T

## Análisis y visualización de los datos

Los datos que se encuentran en el archivo comprimidos `meli-challenge-2019.tar.bz2` constan de 8 archivos, 4 en español y 4 en portugués.

Los 4 archivos en españos son: Un conjunto de datos o set de entrenamiento, un set de validación, un set de prueba y un registro de tokens según la siguiente descripción.

El **set de entrenamiento** original tiene 4895280 registros con valores no nulos y 10 columnas. Las columnas de dicho dataset son:
- **language**: El idioma del dataset (españor o portugués). En el trabajo práctico utilizaremos solamente el dataset es español.
- **label_quality**: Calidad de la etiqueta (confiable o no confiable). Se dispone de 4508043 registros no confiables y 387237 registros confiables.
- **title**: El título que se asignó al producto. **Esta información es la que se utilizará para armar el dataser de entrenamiento.**
- **category**: La categoría que se asignó al producto. **Este es el target**.
- **split**: El tipo de dataset. _train_ para el set de entrenamiento.
- **tokenized_title**: El título tokenizado. Esto significa que los datos fueron preprocesados.
- **data**: El número asignado a cada palabra del título tokenizado.
- **target**: El número que corresponde a cada categoría.
- **n_labels**: Cantidad de etiquetas numéricas correspondientes a las distintas categorías. 632 etiquetas (0 a 631) para el caso del set de entrenamiento.
- **size**: La cantidad de registros. 4895280 registros para el caso del set de entrenamiento.

El **set de prueba** original tiene 63680 registros con valores no nulos y 10 columnas. Las columnas de dicho dataset son:
- **language**: El idioma del dataset (españor o portugués). En el trabajo práctico utilizaremos solamente el dataset es español.
- **label_quality**: Calidad de la etiqueta (confiable o no confiable). Todas las etiquetas de este dataset son confiables.
- **title**: El título que se asignó al producto.
- **category**: La categoría que se asignó al producto.
- **split**: El tipo de dataset. _test_ para el set de prueba.
- **tokenized_title**: El título tokenizado. Esto significa que los datos fueron preprocesados.
- **data**: El número asignado a cada palabra del título tokenizado.
- **target**: El número que corresponde a cada categoría.
- **n_labels**: Cantidad de etiquetas numéricas correspondientes a las distintas categorías. 632 etiquetas (0 a 631) para el caso del set de prueba.
- **size**: La cantidad de registros. 63680 registros para el caso del set de prueba.

El **set de validación** original tiene 1223820 registros con valores no nulos y 10 columnas. Las columnas de dicho dataset son:
- **language**: El idioma del dataset (españor o portugués). En el trabajo práctico utilizaremos solamente el dataset es español.
- **label_quality**: Calidad de la etiqueta (confiable o no confiable). Se dispone de 1127189 registros no confiables y 96631 registros confiables.
- **title**: El título que se asignó al producto.
- **category**: La categoría que se asignó al producto.
- **split**: El tipo de dataset. _validation_ para el set de prueba.
- **tokenized_title**: El título tokenizado. Esto significa que los datos fueron preprocesados.
- **data**: El número asignado a cada palabra del título tokenizado.
- **target**: El número que corresponde a cada categoría.
- **n_labels**: Cantidad de etiquetas numéricas correspondientes a las distintas categorías. 632 etiquetas (0 a 631) para el caso del set de validación.
- **size**: La cantidad de registros. 1223820 registros para el caso del set de validación.

El archivo **spanish_token_to_index** tiene las 50002 correspondencias que existen entre las palabras tokenizadas del título y las etiquetas numéricas bajo la columna data en los sets de entrenamiento, prueba y validación. No se utilizará este tokenizador, en lugar de ello se utilizará ...

**En este trabajo práctico se utiliza:**
- El **set de entrenamiento** para entrenar el modelo
- El **set de validación** para evaluar el modelo y ajustar hiperparámetros
- El **set de prueba** para mostrar el mejor modelo obtenido

In [6]:
df.head()

,language,label_quality,title,category,split,tokenized_title,data,target,n_labels,size
0,spanish,unreliable,Metal Biela Dw10 Hdi 2.0,ENGINE_BEARINGS,validation,"[metal, biela, hdi]","[457, 1480, 3450]",88,632,1223820
1,spanish,unreliable,Repuestos Martillo Rotoprcutor Bosch Gshsce Po...,ELECTRIC_DEMOLITION_HAMMERS,validation,"[repuestos, martillo, rotoprcutor, bosch, gshs...","[3119, 892, 1, 767, 1, 9337]",174,632,1223820
2,spanish,unreliable,Pesca Caña Pejerrey Colony Brava 3m Fibra De V...,FISHING_RODS,validation,"[pesca, caña, pejerrey, colony, brava, fibra, ...","[700, 990, 2057, 3990, 3670, 1737, 1153, 6568]",313,632,1223820
3,spanish,unreliable,Porcelanato Abitare Be 20x120 Cm. Ceramica Por...,PORCELAIN_TILES,validation,"[porcelanato, abitare, ceramica, portinari]","[2722, 4404, 1406, 4405]",427,632,1223820
4,spanish,unreliable,Reconstruction Semi Di Lino Alfaparf Shampoo 1...,HAIR_SHAMPOOS_AND_CONDITIONERS,validation,"[reconstruction, semi, lino, alfaparf, shampoo]","[1, 3365, 7502, 10919, 849]",194,632,1223820


### Tokens y sus etiquetas

Las siguientes 3 celdas de código demuestran que la relación entre los datos bajo las columnas `tokenized_title` y `data` está dada en el archivo `spanish_token_to_index` que vincula cada palabra a un índice numérico entero. De todos modos, esto es a modo informativo.

In [ ]:
i = 0 # Un índice cualquiera para extraer datos
items = df.at[i, 'tokenized_title']
items

In [ ]:
# Comparar la salida de esta celda con la de la siguiente
df.at[i, 'data']

In [ ]:
nro_items = []
for item in items:
    id_item = tokens.loc[item][0]
    nro_items.append(id_item)
nro_items

Para el presente trabajo práctico solamente interesan las columnas **title** y **category**.

In [8]:
df.drop(columns=['language', 'label_quality', 'split', 'tokenized_title', 'data', 'target', 'n_labels', 'size'],
        inplace=True)
df.head()

,title,category
0,Metal Biela Dw10 Hdi 2.0,ENGINE_BEARINGS
1,Repuestos Martillo Rotoprcutor Bosch Gshsce Po...,ELECTRIC_DEMOLITION_HAMMERS
2,Pesca Caña Pejerrey Colony Brava 3m Fibra De V...,FISHING_RODS
3,Porcelanato Abitare Be 20x120 Cm. Ceramica Por...,PORCELAIN_TILES
4,Reconstruction Semi Di Lino Alfaparf Shampoo 1...,HAIR_SHAMPOOS_AND_CONDITIONERS


In [20]:
# En caso de quererlo, se reduce la muestra a n items
n = 1000000
df = df.sample(n, axis=0)

## Reducción de datos.

Como los datos que interesan son una fracción de los contenidos en el `meli-challenge-2019.tar.bz2`, los mismos se guardan en los siguientes archivos para ser utilizados en la parte 2.

- training_set1.csv
- training_set2.csv
- training_set3.csv
- training_set4.csv
- test_set.csv
- validation_set-csv

In [3]:
nombre_archivo = './data/validation_set.csv'

In [9]:
df.columns

Index(['title', 'category'], dtype='object')

In [11]:
df.to_csv(nombre_archivo, header=list(df.columns), index=False)